In [2]:
import pandas as pd
import torch
import random
import os
from tqdm import tqdm # 导入 tqdm 库

# 定义实体类型前缀，用于ID映射和区分
ENTITY_TYPES = {
    'user': 'U_',
    'course': 'C_',
    'knowledge': 'K_',
    'school': 'S_',
    'teacher': 'T_',
    'major': 'M_'
}

def load_and_process_data_for_experiment(
    data_dir, # 数据文件所在的目录
    # 消融实验参数
    include_kg_course_knowledge=True, 
    include_kg_school_course=True,
    include_kg_teacher_course=True,
    include_kg_kp_prereq=True,
    include_kg_knowledge_major=True
):
    """
    加载所有原始数据，进行全局ID映射，划分训练/测试交互，并根据配置构建图。
    在耗时部分添加了tqdm进度条。
    """
    # --- 1. 加载所有原始数据 ---
    print("Step 1: Loading raw data files...")
    df_train_uc = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df_test_uc = pd.read_csv(os.path.join(data_dir, 'test.csv'))
    df_knowledge_major = pd.read_csv(os.path.join(data_dir, 'knowledge-major.csv'))
    df_course_knowledge = pd.read_csv(os.path.join(data_dir, 'course-knowledge.csv'))
    df_school_course = pd.read_csv(os.path.join(data_dir, 'school-course.csv'))
    df_teacher_course = pd.read_csv(os.path.join(data_dir, 'teacher-course.csv'))
    df_prerequisite_relations = pd.read_csv(os.path.join(data_dir, 'prerequisite_relations.csv'))
    print("Data files loaded successfully.")

    # --- 2. 收集所有唯一实体并分配全局ID ---
    print("\nStep 2: Building global entity-to-ID mapping...")
    entity_to_id = {}
    id_counter = 0

    def get_global_id(entity_str):
        nonlocal id_counter
        if entity_str not in entity_to_id:
            entity_to_id[entity_str] = id_counter
            id_counter += 1
        return entity_to_id[entity_str]

    # 使用tqdm来显示实体收集的进度
    all_entities = [
        ("Users", pd.concat([df_train_uc['user'], df_test_uc['user']]).unique()),
        ("Courses", pd.concat([df_train_uc['course'], df_test_uc['course'], 
                               df_course_knowledge['course'], df_school_course['course'], 
                               df_teacher_course['course']]).unique()),
        ("Knowledge Points", pd.concat([df_knowledge_major['knowledge'], df_course_knowledge['knowledge'], 
                                        df_prerequisite_relations['knowledge1'], df_prerequisite_relations['knowledge2']]).unique()),
        ("Schools", df_school_course['school'].unique()),
        ("Teachers", df_teacher_course['teacher'].unique()),
        ("Majors", df_knowledge_major['major'].unique())
    ]
    
    for name, entities in all_entities:
        for entity in tqdm(entities, desc=f"  Processing {name}"):
            get_global_id(entity)

    num_nodes = id_counter # 总节点数
    print(f"Global ID mapping built. Total unique nodes: {num_nodes}")

    # 记录用户和课程的全局ID范围
    user_ids = sorted([v for k,v in entity_to_id.items() if k.startswith(ENTITY_TYPES['user'])])
    course_ids = sorted([v for k,v in entity_to_id.items() if k.startswith(ENTITY_TYPES['course'])])
    user_ids_global_range = (min(user_ids), max(user_ids) + 1) if user_ids else (0,0)
    course_ids_global_range = (min(course_ids), max(course_ids) + 1) if course_ids else (0,0)

    # --- 3. 构建训练集和测试集的用户-课程交互（已ID化） ---
    print("\nStep 3: Processing train and test interactions...")
    train_interactions = []
    user_positive_items_train = {}
    for _, row in tqdm(df_train_uc.iterrows(), total=df_train_uc.shape[0], desc="  Processing train.csv"):
        u_id = get_global_id(row['user'])
        c_id = get_global_id(row['course'])
        train_interactions.append((u_id, c_id))
        user_positive_items_train.setdefault(u_id, set()).add(c_id)

    test_interactions = []
    user_positive_items_test = {}
    for _, row in tqdm(df_test_uc.iterrows(), total=df_test_uc.shape[0], desc="  Processing test.csv"):
        u_id = get_global_id(row['user'])
        c_id = get_global_id(row['course'])
        test_interactions.append((u_id, c_id))
        user_positive_items_test.setdefault(u_id, []).append(c_id)
    print("Interactions processed.")

    # --- 4. 构建用于LightGCN的图的 edge_index ---
    print("\nStep 4: Building graph edges...")
    edges = []
    # 4.1 添加训练集的用户-课程交互边 (双向)
    for u_id, c_id in tqdm(train_interactions, desc="  Adding user-course edges"):
        edges.append((u_id, c_id))
        edges.append((c_id, u_id))

    # 4.2 根据消融实验配置添加知识图谱边 (双向)
    if include_kg_course_knowledge:
        for _, row in tqdm(df_course_knowledge.iterrows(), total=df_course_knowledge.shape[0], desc="  Adding course-knowledge edges"):
            c_id = get_global_id(row['course'])
            k_id = get_global_id(row['knowledge'])
            edges.append((c_id, k_id))
            edges.append((k_id, c_id))

    if include_kg_school_course:
        for _, row in tqdm(df_school_course.iterrows(), total=df_school_course.shape[0], desc="  Adding school-course edges"):
            s_id = get_global_id(row['school'])
            c_id = get_global_id(row['course'])
            edges.append((s_id, c_id))
            edges.append((c_id, s_id))

    if include_kg_teacher_course:
        for _, row in tqdm(df_teacher_course.iterrows(), total=df_teacher_course.shape[0], desc="  Adding teacher-course edges"):
            t_id = get_global_id(row['teacher'])
            c_id = get_global_id(row['course'])
            edges.append((t_id, c_id))
            edges.append((c_id, t_id))

    if include_kg_kp_prereq:
        for _, row in tqdm(df_prerequisite_relations.iterrows(), total=df_prerequisite_relations.shape[0], desc="  Adding prerequisite edges"):
            kp1_id = get_global_id(row['knowledge1'])
            kp2_id = get_global_id(row['knowledge2'])
            edges.append((kp1_id, kp2_id))
            edges.append((kp2_id, kp1_id))

    if include_kg_knowledge_major:
        for _, row in tqdm(df_knowledge_major.iterrows(), total=df_knowledge_major.shape[0], desc="  Adding knowledge-major edges"):
            k_id = get_global_id(row['knowledge'])
            m_id = get_global_id(row['major'])
            edges.append((k_id, m_id))
            edges.append((m_id, k_id))
            
    print("Graph edges built. Converting to tensor...")
    src_nodes, dst_nodes = zip(*edges)
    edge_index = torch.tensor([src_nodes, dst_nodes], dtype=torch.long)
    print("Data loading and processing complete.")

    return num_nodes, edge_index, train_interactions, test_interactions, \
           user_ids_global_range, course_ids_global_range, \
           user_positive_items_train, user_positive_items_test, entity_to_id

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 从 PyTorch Geometric 导入辅助函数
from torch_geometric.utils import add_self_loops, degree

class LightGCN(nn.Module):
    """
    LightGCN模型实现。
    LightGCN通过移除GCN中的特征变换和非线性激活函数，简化图卷积过程。
    它在异构图上进行消息传播，学习所有节点的Embedding。
    """
    def __init__(self, num_nodes, embedding_dim, num_layers):
        """
        初始化LightGCN模型。

        Args:
            num_nodes (int): 图中所有节点的总数（用户+课程+知识点+学校+老师+专业）。
            embedding_dim (int): 节点Embedding的维度。
            num_layers (int): LightGCN的消息传播层数。
        """
        super(LightGCN, self).__init__()
        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        # 为所有节点初始化Embedding向量
        self.embedding = nn.Embedding(num_nodes, embedding_dim)
        # 使用Xavier均匀分布初始化Embedding权重，有助于训练稳定
        nn.init.xavier_uniform_(self.embedding.weight)

    def forward(self, edge_index):
        """
        执行LightGCN的消息传播过程。

        Args:
            edge_index (torch.LongTensor): 图的边索引，格式为 (2, num_edges)。
                                            包含了所有异构边。

        Returns:
            torch.Tensor: 所有节点的最终学习到的Embedding矩阵，形状为 (num_nodes, embedding_dim)。
        """
        # 1. 添加自循环并标准化邻接矩阵 (LightGCN的A_hat = A + I)
        # add_self_loops 函数会向 edge_index 添加自循环，并返回新的边索引。
        edge_index_norm, _ = add_self_loops(edge_index, num_nodes=self.num_nodes)

        # 计算度矩阵的逆平方根，用于归一化
        row, col = edge_index_norm
        deg = degree(col, self.num_nodes, dtype=row.dtype) # 计算每个节点的度
        deg_inv_sqrt = deg.pow(-0.5) # 度^-0.5
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0 # 避免对度为0的节点进行无穷大操作

        # 计算归一化因子 (D_hat)^(-1/2) * (D_hat)^(-1/2)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # 构建稀疏邻接矩阵。PyTorch Geometric在内部会更高效地处理图结构。
        # 这里手动构建是为了更清晰地展示LightGCN的核心传播步骤。
        adj_matrix = torch.sparse_coo_tensor(
            edge_index_norm, norm, (self.num_nodes, self.num_nodes), 
            dtype=torch.float32, device=self.embedding.weight.device
        )
        
        # 2. 消息传播
        # all_embeddings 列表存储每一层传播后的节点Embedding
        all_embeddings = [self.embedding.weight] # L0 层的Embedding就是初始Embedding

        for layer in range(self.num_layers):
            # E^(l+1) = (Normalized_Adj_Matrix) * E^(l)
            # 即每个节点从其邻居聚合信息
            current_embeddings = torch.sparse.mm(adj_matrix, all_embeddings[-1])
            all_embeddings.append(current_embeddings)

        # 3. 最终Embedding聚合 (所有层级Embedding的平均)
        # LightGCN的最终Embedding是所有层（包括初始层）Embedding的平均。
        # torch.stack 将列表中的张量沿新维度堆叠起来 (num_layers+1, num_nodes, embedding_dim)
        # torch.mean 沿第一个维度（层维度）取平均，得到 (num_nodes, embedding_dim)
        final_embeddings = torch.mean(torch.stack(all_embeddings, dim=1), dim=1)
        
        return final_embeddings

    # 辅助方法：在评估阶段，可以根据全局ID范围获取特定类型的Embedding
    # 通常不需要直接调用，因为评估函数直接通过全局ID索引 final_embeddings
    def get_user_course_embeddings(self, final_embeddings, user_ids_global_range, course_ids_global_range):
        user_embeddings = final_embeddings[user_ids_global_range[0]:user_ids_global_range[1]]
        course_embeddings = final_embeddings[course_ids_global_range[0]:course_ids_global_range[1]]
        return user_embeddings, course_embeddings

In [4]:
import torch
import torch.nn.functional as F
import numpy as np
import random
from tqdm import tqdm

def train_model(model, edge_index, train_interactions,
                user_ids_global_range, course_ids_global_range,
                num_epochs, batch_size, learning_rate, device):
    """
    训练LightGCN模型，并返回每个Epoch的损失。
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.to(device)
    edge_index = edge_index.to(device)

    all_course_global_ids = list(range(course_ids_global_range[0], course_ids_global_range[1]))
    
    print("Building training user-positive-items map...")
    user_positive_items = {}
    for u_id, c_id in tqdm(train_interactions, desc="  Building map"):
        user_positive_items.setdefault(u_id, set()).add(c_id)

    # 新增：用于记录每个Epoch的损失
    epoch_losses = []

    print("Starting training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        random.shuffle(train_interactions)
        
        # 遍历训练交互数据，按批次进行训练
        batch_iterator = tqdm(range(0, len(train_interactions), batch_size), desc=f"Epoch {epoch+1}")
        for i in batch_iterator:
            batch_interactions = train_interactions[i : i + batch_size]
            
            batch_users, batch_pos_courses, batch_neg_courses = [], [], []
            for u_id, pos_c_id in batch_interactions:
                batch_users.append(u_id)
                batch_pos_courses.append(pos_c_id)
                neg_c_id = random.choice(all_course_global_ids)
                while neg_c_id in user_positive_items.get(u_id, set()):
                    neg_c_id = random.choice(all_course_global_ids)
                batch_neg_courses.append(neg_c_id)

            batch_users_tensor = torch.tensor(batch_users, dtype=torch.long).to(device)
            batch_pos_courses_tensor = torch.tensor(batch_pos_courses, dtype=torch.long).to(device)
            batch_neg_courses_tensor = torch.tensor(batch_neg_courses, dtype=torch.long).to(device)

            optimizer.zero_grad()
            
            final_embeddings = model(edge_index)
            
            user_embeddings = final_embeddings[batch_users_tensor]
            pos_course_embeddings = final_embeddings[batch_pos_courses_tensor]
            neg_course_embeddings = final_embeddings[batch_neg_courses_tensor]

            pos_scores = (user_embeddings * pos_course_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_course_embeddings).sum(dim=1)
            loss = -F.logsigmoid(pos_scores - neg_scores).mean()
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            # 在tqdm进度条上动态显示当前批次的损失
            batch_iterator.set_postfix(loss=loss.item())
        
        avg_epoch_loss = total_loss / (len(train_interactions) / batch_size)
        epoch_losses.append(avg_epoch_loss)
        print(f"Epoch {epoch+1}, Average Loss: {avg_epoch_loss:.4f}")
    
    print("Training finished.")

    # 返回模型、最终Embedding和损失历史
    return model, final_embeddings, epoch_losses

# evaluate_model 函数保持不变
# ... (将 evaluate_model 函数的完整代码粘贴到这里) ...
def evaluate_model(model, edge_index, final_embeddings, 
                   test_interactions,
                   user_ids_global_range, course_ids_global_range,
                   user_positive_items_train, # 用户在训练集中已交互的课程
                   user_positive_items_test,  # 用户在测试集中已交互的课程
                   k_values=[10, 20], num_neg_samples=99, device='cpu'):
    model.eval() # 设置模型为评估模式 (禁用 dropout 等)
    
    # 获取所有课程的全局ID列表，用于负采样
    course_start_id, course_end_id = course_ids_global_range
    all_course_global_ids = list(range(course_start_id, course_end_id))

    # 获取在测试集中有实际交互的用户列表，只评估这些用户
    test_users = list(user_positive_items_test.keys())
    
    hr_scores = {k: [] for k in k_values}
    ndcg_scores = {k: [] for k in k_values}

    print("Starting evaluation...")
    with torch.no_grad(): # 在评估阶段不需要计算梯度，节省内存和时间
        
        for u_id in tqdm(test_users, desc="Evaluating"):
            # 1. 获取该用户在测试集中的所有真实正样本
            positive_courses_in_test = user_positive_items_test.get(u_id, [])
            if not positive_courses_in_test: 
                continue # 如果用户在测试集中没有正样本，则跳过

            # 2. 获取用户在训练集中已交互过的所有课程 (用于负样本排除)
            interacted_courses_train = user_positive_items_train.get(u_id, set())
            
            # 3. 构建候选推荐列表：包含测试集正样本 + 负样本
            candidate_courses_global_ids = []
            candidate_courses_global_ids.extend(positive_courses_in_test) # 加入测试集中的所有正样本

            # 采样负样本
            neg_candidates = []
            # 用户所有已知的交互 (训练集正样本 + 测试集正样本)
            current_user_all_interacted_courses = interacted_courses_train.union(set(positive_courses_in_test))

            while len(neg_candidates) < num_neg_samples:
                sampled_neg_id = random.choice(all_course_global_ids) # 从所有课程中随机选
                # 确保采样到的负样本：不在用户所有已交互课程中
                if sampled_neg_id not in current_user_all_interacted_courses:
                    neg_candidates.append(sampled_neg_id)
            
            candidate_courses_global_ids.extend(neg_candidates)
            random.shuffle(candidate_courses_global_ids) # 打乱顺序，避免任何偏差

            # 将候选课程IDs转换为PyTorch Tensor，并移动到指定设备
            candidate_courses_tensor = torch.tensor(candidate_courses_global_ids, dtype=torch.long).to(device)

            # 4. 预测分数
            user_emb = final_embeddings[u_id].unsqueeze(0) # (1, embedding_dim)
            candidate_embs = final_embeddings[candidate_courses_tensor] # (num_candidates, embedding_dim)
            
            scores = torch.matmul(user_emb, candidate_embs.T).squeeze(0) # (num_candidates,)

            # 5. 排序并获取Top-K推荐列表
            _, top_indices = torch.topk(scores, k=max(k_values))
            predicted_global_ids = candidate_courses_tensor[top_indices].cpu().numpy()

            # 6. 计算HR和NDCG
            for k in k_values:
                # HR@K (Hit Ratio)
                hit = False
                for pos_c_id in positive_courses_in_test:
                    if pos_c_id in predicted_global_ids[:k]: 
                        hit = True
                        break 
                hr_scores[k].append(1 if hit else 0)

                # NDCG@K (Normalized Discounted Cumulative Gain)
                dcg = 0.0 # Discounted Cumulative Gain
                
                # 计算DCG：遍历Top-K推荐列表
                for rank, pred_id in enumerate(predicted_global_ids[:k]):
                    if pred_id in positive_courses_in_test:
                        dcg += 1.0 / np.log2(rank + 2) 
                
                # 计算IDCG (Ideal DCG)：理想情况下的最大DCG
                idcg = 0.0
                num_relevant_in_k = min(len(positive_courses_in_test), k)
                for rank in range(num_relevant_in_k):
                    idcg += 1.0 / np.log2(rank + 2)

                if idcg == 0: 
                    ndcg_scores[k].append(0.0)
                else:
                    ndcg_scores[k].append(dcg / idcg)
    
    # 7. 汇总所有用户的指标
    avg_hr = {k: np.mean(hr_scores[k]) for k in k_values}
    avg_ndcg = {k: np.mean(ndcg_scores[k]) for k in k_values}

    print("\nEvaluation Results:")
    for k in k_values:
        print(f"HR@{k}: {avg_hr[k]:.4f}")
        print(f"NDCG@{k}: {avg_ndcg[k]:.4f}")

    return avg_hr, avg_ndcg

In [ ]:
# 版本1 全部实验未调参

import torch
import os
import sys
import pandas as pd
import json



if __name__ == "__main__":
    # 检查是否有可用的GPU，并设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 定义数据文件所在的目录
    data_dir = '/kaggle/input/data12/data/' 
    results_dir = '/kaggle/working/data12results/'
    os.makedirs(results_dir, exist_ok=True) # 确保结果目录存在
    # 2. 模型和训练超参数
    embedding_dim = 32  # 节点Embedding的维度
    num_layers = 8      # LightGCN的消息传播层数
    num_epochs = 50     # 训练轮数
    batch_size = 2048   # BPR损失计算的批处理大小
    learning_rate = 0.001 # 优化器学习率

    # --- 消融实验的配置字典 ---
    experiment_configs = {
        "Baseline_LightGCN": { # 仅用户-课程交互
            "include_kg_course_knowledge": False,
            "include_kg_school_course": False,
            "include_kg_teacher_course": False,
            "include_kg_kp_prereq": False,
            "include_kg_knowledge_major": False
        },
        "LightGCN_plus_CourseKnowledge": { # 加入课程-知识点
            "include_kg_course_knowledge": True,
            "include_kg_school_course": False,
            "include_kg_teacher_course": False,
            "include_kg_kp_prereq": False,
            "include_kg_knowledge_major": False
        },
        "LightGCN_plus_CourseKnowledge_School": { # 加入学校-课程
            "include_kg_course_knowledge": True,
            "include_kg_school_course": True,
            "include_kg_teacher_course": False,
            "include_kg_kp_prereq": False,
            "include_kg_knowledge_major": False
        },
        "LightGCN_plus_CourseKnowledge_School_Teacher": { # 加入老师-课程
            "include_kg_course_knowledge": True,
            "include_kg_school_course": True,
            "include_kg_teacher_course": True,
            "include_kg_kp_prereq": False,
            "include_kg_knowledge_major": False
        },
        "LightGCN_plus_CourseKnowledge_School_Teacher_KPMajor": { # 加入知识点-专业
            "include_kg_course_knowledge": True,
            "include_kg_school_course": True,
            "include_kg_teacher_course": True,
            "include_kg_kp_prereq": False,
            "include_kg_knowledge_major": True
        },
        "LightGCN_Full_KG": { # 最终模型 (包含所有知识图谱边)
            "include_kg_course_knowledge": True,
            "include_kg_school_course": True,
            "include_kg_teacher_course": True,
            "include_kg_kp_prereq": True,
            "include_kg_knowledge_major": True
        }
    }

    results_summary = {} # 用于存储所有实验的最终结果

    # --- 控制运行单个实验的开关 ---
    # 将此变量设置为你希望运行的实验名称（键），例如 "LightGCN_Full_KG"
    # 如果设置为 None 或 ""，则会运行 'experiment_configs' 中定义的所有实验。
    # run_specific_experiment = "LightGCN_Full_KG" # 默认先跑完整模型
    # run_specific_experiment = "Baseline_LightGCN" # 也可以先跑Baseline
    run_specific_experiment = None # 设置为 None 会运行所有实验

    if run_specific_experiment:
        print(f"Running ONLY the experiment: {run_specific_experiment}\n")
    else:
        print("Running ALL defined experiments.\n")

    for exp_name, config in experiment_configs.items():
        if run_specific_experiment and exp_name != run_specific_experiment:
            continue

        print(f"\n--- Running Experiment: {exp_name} ---")
        
        # 1. 数据加载和处理
        num_nodes, edge_index, train_interactions, test_interactions, \
        user_ids_global_range, course_ids_global_range, \
        user_positive_items_train, user_positive_items_test, entity_to_id = \
            load_and_process_data_for_experiment(data_dir, **config)

        print(f"Graph nodes: {num_nodes}, edges: {edge_index.shape[1] // 2}")
        print(f"Training interactions: {len(train_interactions)}, Test interactions: {len(test_interactions)}")

        # 3. 初始化模型
        model = LightGCN(num_nodes, embedding_dim, num_layers)
        print(f"Model initialized for {exp_name}.")

        # 4. 训练模型
        trained_model, final_embeddings, epoch_losses = train_model(
            model, edge_index, train_interactions,
            user_ids_global_range, course_ids_global_range,
            num_epochs, batch_size, learning_rate, device
        )

        # --- 保存训练过程和结果 ---
        # 创建当前实验的结果子目录
        exp_results_dir = os.path.join(results_dir, exp_name)
        os.makedirs(exp_results_dir, exist_ok=True)
        
        # 4.1 保存模型权重
        model_save_path = os.path.join(exp_results_dir, 'model_weights.pt')
        torch.save(trained_model.state_dict(), model_save_path)
        print(f"Model weights for '{exp_name}' saved to {model_save_path}")

        # 4.2 保存损失历史
        loss_df = pd.DataFrame({'epoch': range(1, num_epochs + 1), 'loss': epoch_losses})
        loss_save_path = os.path.join(exp_results_dir, 'loss_history.csv')
        loss_df.to_csv(loss_save_path, index=False)
        print(f"Loss history for '{exp_name}' saved to {loss_save_path}")
        
        # 4.3 保存最终Embedding
        embedding_save_path = os.path.join(exp_results_dir, 'final_embeddings.pt')
        torch.save(final_embeddings, embedding_save_path)
        print(f"Final embeddings for '{exp_name}' saved to {embedding_save_path}")
        
        # 4.4 保存ID映射 (非常重要，用于解释Embedding)
        id_map_save_path = os.path.join(exp_results_dir, 'entity_to_id.json')
        with open(id_map_save_path, 'w', encoding='utf-8') as f:
            json.dump(entity_to_id, f, ensure_ascii=False, indent=4)
        print(f"Entity-to-ID map for '{exp_name}' saved to {id_map_save_path}")

        # 5. 评估模型
        hr_results, ndcg_results = evaluate_model(
            trained_model, edge_index, final_embeddings, 
            test_interactions,
            user_ids_global_range, course_ids_global_range,
            user_positive_items_train, user_positive_items_test,
            k_values=[10, 20], device=device
        )
        
        results_summary[exp_name] = {
            "HR@10": hr_results[10], "NDCG@10": ndcg_results[10],
            "HR@20": hr_results[20], "NDCG@20": ndcg_results[20]
        }
    
    # 打印并保存所有实验的最终结果汇总
    print("\n\n--- All Experiments Summary ---")
    if not results_summary:
        print("No experiments were run.")
    else:
        summary_df = pd.DataFrame.from_dict(results_summary, orient='index')
        summary_df.index.name = 'Experiment_Name'
        print(summary_df)
        
        # 保存汇总结果到CSV文件
        summary_save_path = os.path.join(results_dir, 'experiment_summary.csv')
        summary_df.to_csv(summary_save_path)
        print(f"\nExperiment summary saved to {summary_save_path}")

In [6]:
#版本2 网格调参
import torch
import os
import sys
import pandas as pd
import json
import itertools # 导入itertools来轻松生成笛卡尔积


if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 定义数据文件和结果输出的目录
    data_dir = '/kaggle/input/data12/data/' 
    results_dir = '/kaggle/working/hyperparameter_tuning_results/' # 为调优结果创建一个新目录
    os.makedirs(results_dir, exist_ok=True) # 确保结果目录存在

    # --- 1. 定义超参数网格 ---
    param_grid = {
        'num_layers': [2, 3, 4],
        'embedding_dim': [32, 64, 128], # 为了节省时间，先不跑256，如果128效果好再加
        'learning_rate': [0.01, 0.005, 0.001]
    }
    
    # 固定的训练参数
    num_epochs = 50
    batch_size = 2048

    # --- 2. 准备数据加载所需的固定配置 ---
    # 我们只对 Full_KG 模型进行调优
    full_kg_config = {
        "include_kg_course_knowledge": True,
        "include_kg_school_course": True,
        "include_kg_teacher_course": True,
        "include_kg_kp_prereq": True,
        "include_kg_knowledge_major": True
    }
    
    # --- 3. 数据只加载一次，因为图结构是固定的 (Full KG) ---
    print("--- Loading and Processing Data for Full KG Model (once) ---")
    num_nodes, edge_index, train_interactions, test_interactions, \
    user_ids_global_range, course_ids_global_range, \
    user_positive_items_train, user_positive_items_test, entity_to_id = \
        load_and_process_data_for_experiment(data_dir, **full_kg_config)
    
    print(f"Data loaded. Graph nodes: {num_nodes}, edges: {edge_index.shape[1] // 2}")
    
    # --- 4. 网格搜索主循环 ---
    results_summary = {}
    
    # 生成所有超参数组合的笛卡尔积
    param_combinations = list(itertools.product(
        param_grid['num_layers'],
        param_grid['embedding_dim'],
        param_grid['learning_rate']
    ))
    
    total_experiments = len(param_combinations)
    print(f"\n--- Starting Grid Search for {total_experiments} Hyperparameter Combinations ---")

    for i, (num_layers, embedding_dim, learning_rate) in enumerate(param_combinations):
        
        # 动态生成实验名称
        exp_name = f"L{num_layers}_D{embedding_dim}_LR{learning_rate}"
        
        print(f"\n--- Running Experiment {i+1}/{total_experiments}: {exp_name} ---")
        
        # 4.1 初始化模型 (每次循环都重新初始化)
        model = LightGCN(num_nodes, embedding_dim, num_layers)
        print(f"Model initialized: layers={num_layers}, dim={embedding_dim}, lr={learning_rate}")

        # 4.2 训练模型
        trained_model, final_embeddings, epoch_losses = train_model(
            model, edge_index, train_interactions,
            user_ids_global_range, course_ids_global_range,
            num_epochs, batch_size, learning_rate, device
        )

        # 4.3 保存训练过程和结果
        exp_results_dir = os.path.join(results_dir, exp_name)
        os.makedirs(exp_results_dir, exist_ok=True)
        
        # 保存模型权重
        model_save_path = os.path.join(exp_results_dir, 'model_weights.pt')
        torch.save(trained_model.state_dict(), model_save_path)
        
        # 保存损失历史
        loss_df = pd.DataFrame({'epoch': range(1, num_epochs + 1), 'loss': epoch_losses})
        loss_save_path = os.path.join(exp_results_dir, 'loss_history.csv')
        loss_df.to_csv(loss_save_path, index=False)
        print(f"Results for '{exp_name}' saved to {exp_results_dir}")

        # 4.4 评估模型
        hr_results, ndcg_results = evaluate_model(
            trained_model, edge_index, final_embeddings, 
            test_interactions,
            user_ids_global_range, course_ids_global_range,
            user_positive_items_train, user_positive_items_test,
            k_values=[10, 20], device=device
        )
        
        # 4.5 记录结果到汇总字典
        results_summary[exp_name] = {
            "num_layers": num_layers,
            "embedding_dim": embedding_dim,
            "learning_rate": learning_rate,
            "HR@10": hr_results[10], "NDCG@10": ndcg_results[10],
            "HR@20": hr_results[20], "NDCG@20": ndcg_results[20]
        }
    
    # 5. 打印并保存所有实验的最终结果汇总
    print("\n\n--- Grid Search Summary ---")
    if not results_summary:
        print("No experiments were run.")
    else:
        summary_df = pd.DataFrame.from_dict(results_summary, orient='index')
        summary_df = summary_df.sort_values(by="NDCG@10", ascending=False) # 按NDCG@10降序排序
        print(summary_df)
        
        # 保存汇总结果到CSV文件
        summary_save_path = os.path.join(results_dir, 'grid_search_summary.csv')
        summary_df.to_csv(summary_save_path)
        print(f"\nGrid search summary saved to {summary_save_path}")

In [7]:
# 版本3 探索网络深度
import torch
import os
import sys
import pandas as pd
import json
import itertools


if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 定义数据文件和结果输出的目录
    data_dir = '/kaggle/input/data12/data/'
    # 为深度探索实验创建一个新的结果目录
    results_dir = '/kaggle/working/depth_exploration_results/'
    os.makedirs(results_dir, exist_ok=True)

    # --- 1. 定义深度探索的参数 ---
    # 根据你之前的网格搜索结果，我们找到了最优的 embedding_dim 和 learning_rate
    best_embedding_dim = 32
    best_learning_rate = 0.001
    
    # 现在，我们只探索 num_layers 的变化
    depth_exploration_grid = {
        'num_layers': [14,16], 
        'embedding_dim': [best_embedding_dim], # 固定为最优值
        'learning_rate': [best_learning_rate] # 固定为最优值
    }
    
    # 固定的训练参数
    num_epochs = 50
    batch_size = 2048 # 使用你之前调参时固定的batch_size

    # --- 2. 准备数据加载所需的固定配置 (只在Full KG模型上探索) ---
    full_kg_config = {
        "include_kg_course_knowledge": True, "include_kg_school_course": True,
        "include_kg_teacher_course": True, "include_kg_kp_prereq": True,
        "include_kg_knowledge_major": True
    }
    
    # --- 3. 数据只加载一次 ---
    print("--- Loading and Processing Data for Full KG Model (once) ---")
    num_nodes, edge_index, train_interactions, test_interactions, \
    user_ids_global_range, course_ids_global_range, \
    user_positive_items_train, user_positive_items_test, entity_to_id = \
        load_and_process_data_for_experiment(data_dir, **full_kg_config)
    
    print(f"Data loaded. Graph nodes: {num_nodes}, edges: {edge_index.shape[1] // 2}")
    
    # --- 4. 深度探索主循环 ---
    results_summary = {}
    
    # 生成所有超参数组合 (这里实际上只是遍历num_layers)
    param_combinations = list(itertools.product(
        depth_exploration_grid['num_layers'],
        depth_exploration_grid['embedding_dim'],
        depth_exploration_grid['learning_rate']
    ))
    
    total_experiments = len(param_combinations)
    print(f"\n--- Starting Depth Exploration for {total_experiments} Configurations ---")

    for i, (num_layers, embedding_dim, learning_rate) in enumerate(param_combinations):
        
        # 动态生成实验名称
        exp_name = f"Layers_{num_layers}" # 实验名称现在只关注层数
        
        print(f"\n--- Running Experiment {i+1}/{total_experiments}: {exp_name} ---")
        
        # 4.1 初始化模型
        model = LightGCN(num_nodes, embedding_dim, num_layers)
        print(f"Model initialized: layers={num_layers}, dim={embedding_dim}, lr={learning_rate}")

        # 4.2 训练模型
        trained_model, final_embeddings, epoch_losses = train_model(
            model, edge_index, train_interactions,
            user_ids_global_range, course_ids_global_range,
            num_epochs, batch_size, learning_rate, device
        )

        # 4.3 保存训练过程和结果
        exp_results_dir = os.path.join(results_dir, exp_name)
        os.makedirs(exp_results_dir, exist_ok=True)
        
        torch.save(trained_model.state_dict(), os.path.join(exp_results_dir, 'model_weights.pt'))
        
        loss_df = pd.DataFrame({'epoch': range(1, num_epochs + 1), 'loss': epoch_losses})
        loss_df.to_csv(os.path.join(exp_results_dir, 'loss_history.csv'), index=False)
        print(f"Results for '{exp_name}' saved to {exp_results_dir}")

        # 4.4 评估模型
        hr_results, ndcg_results = evaluate_model(
            trained_model, edge_index, final_embeddings, 
            test_interactions,
            user_ids_global_range, course_ids_global_range,
            user_positive_items_train, user_positive_items_test,
            k_values=[10, 20], device=device
        )
        
        # 4.5 记录结果到汇总字典
        results_summary[exp_name] = {
            "num_layers": num_layers,
            "HR@10": hr_results[10], "NDCG@10": ndcg_results[10],
            "HR@20": hr_results[20], "NDCG@20": ndcg_results[20]
        }
    
    # 5. 打印并保存所有实验的最终结果汇总
    print("\n\n--- Depth Exploration Summary ---")
    if not results_summary:
        print("No experiments were run.")
    else:
        summary_df = pd.DataFrame.from_dict(results_summary, orient='index')
        # summary_df = summary_df.sort_values(by="num_layers", ascending=True) # 按层数排序
        print(summary_df)
        
        summary_save_path = os.path.join(results_dir, 'depth_exploration_summary.csv')
        summary_df.to_csv(summary_save_path)
        print(f"\nDepth exploration summary saved to {summary_save_path}")